In [1]:
import re
import torch
import utils
from trainer import Trainer, TrainerV2
from model import BertSLU, BertSLUV2, BertSLUV3
from functools import partial
from dataset import BertDataset
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer

c:\Users\lehuy\miniconda3\envs\SLU\lib\site-packages\torch\cuda\__init__.py:651: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [2]:
intent_model = BertSLUV3("intent_class", 15, 9, "vinai/phobert-base-v2")
token_model = BertSLUV3("token_class", 15, 9, "vinai/phobert-base-v2")
params = torch.load('checkpoint/slu_intent.pt',map_location='cpu')
intent_model.load_state_dict(params["model"])
params = torch.load('checkpoint/slu_token.pt',map_location='cpu')
token_model.load_state_dict(params["model"])

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [12]:
sentence = "bật đèn bàn xiaomi ở phòng khách "
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
intent_model.eval()
token_model.eval()
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=64)
inputs = {k: v.to('cpu') for k, v in inputs.items()}
intent_logits = intent_model(inputs)
intent_logits = intent_logits[1].argmax(-1).view(-1).tolist() if intent_logits is not None else [None]
token_logits = token_model(inputs)
token_logits = token_logits[0].argmax(-1).tolist() if token_logits is not None else [None]

In [8]:
INVERSE_MAP_TOKENS = {
    0: 'word',
    1: 'time at',
    2: 'device',
    3: 'changing value',
    4: 'scene',
    5: 'command',
    6: 'location',
    7: 'duration',
    8: 'target number'
 }

INVERSE_MAP_INTENTS = {
    0: 'Giảm độ sáng của thiết bị',
    1: 'Đóng thiết bị',
    2: 'Hủy hoạt cảnh',
    3: 'Tắt thiết bị',
    4: 'Tăng âm lượng của thiết bị',
    5: 'Giảm mức độ của thiết bị',
    6: 'Bật thiết bị',
    7: 'Tăng mức độ của thiết bị',
    8: 'Tăng nhiệt độ của thiết bị',
    9: 'Kiểm tra tình trạng thiết bị',
    10: 'Mở thiết bị',
    11: 'Giảm âm lượng của thiết bị',
    12: 'Kích hoạt cảnh',
    13: 'Giảm nhiệt độ của thiết bị',
    14: 'Tăng độ sáng của thiết bị'
}

In [9]:
INVERSE_MAP_INTENTS[intent_logits[0]]

'Bật thiết bị'

In [10]:
def collect_label(token):
    token = token[1:]
    for i in range(len(token) - 1, -1, -1):
        if token[i] != 0:
            token = token[:i+1]
            break
    token += [-1]
    map_labels = []
    cur = 0
    val = token[0]
    for idx, i in enumerate(token[1:], 1):
        if i == val:
            continue
        else:
            if val != 0:
                map_labels.append([cur, idx-1, val])
            val = i
            cur = idx
    return map_labels

In [11]:
print("INTENTS:",INVERSE_MAP_INTENTS[intent_logits[0]])
token = token_logits[0]
seq = tokenizer.tokenize(sentence)
labels = collect_label(token)
entities = []
for label in labels:
    if label[-1] == 0:
        continue
    sub_text = seq[label[0]: label[1]+1]
    sub_text = tokenizer.decode(
        tokenizer.convert_tokens_to_ids(sub_text), skip_special_tokens=True
    )
    tmp_add = {"type": INVERSE_MAP_TOKENS[label[-1]], "filler": sub_text}
    entities += [tmp_add]
print("ENTITIES:",entities)

INTENTS: Bật thiết bị
ENTITIES: [{'type': 'command', 'filler': 'bật'}, {'type': 'device', 'filler': 'đèn bàn xiami'}, {'type': 'location', 'filler': 'phòng khách'}]
